In [46]:
import pandas as pd
import altair as alt
from IPython.display import display



#https://docs.google.com/spreadsheets/d/1ZMFrD6F6tvPtf_8McC-kWrNBBec_6Si3NW6AoWf3Kbg/edit#gid=1993314609

In [47]:
#automated retrival of google shee from github:https://github.com/tsale/EDR-Telemetry?tab=readme-ov-file

SHEET_ID = '1ZMFrD6F6tvPtf_8McC-kWrNBBec_6Si3NW6AoWf3Kbg'
SHEET_NAME = 'xxdata'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
newdf = pd.read_csv(url)


Preprocesing legends

In [48]:
icons=['🟩','🟧','🟥','🟧','❓','🪵','🎚️']


# check if the 'legend' column contains any of the icons we are interested in
mask = newdf['LEGEND'].isin(icons)



In [49]:
# print the resulting DataFrame, containing only the rows that match the mask
new_df_legend=newdf[mask].reset_index(drop=True)
new_df_legend.dropna(axis=1, inplace=True)
new_df_legend.columns=['value','keys']

In [50]:
new_size=new_df_legend.shape

Preprocesing new spreadsheet

In [51]:
new_df_x=newdf.iloc[6:]

In [52]:
new_df_x.columns = new_df_x.iloc[0]

In [53]:
new_df_x = new_df_x[1:]

In [54]:
new_df_x.reset_index(drop=True, inplace=True)

In [55]:
new_df_x.ffill(inplace=True)

In [56]:
new_df_x.dropna(axis=1,inplace=True)

In [57]:
new_df_x

6,Telemetry Feature Category,Sub-Category,Carbon Black,CrowdStrike,Cybereason,ESET Inspect,Elastic,LimaCharlie,MDE,Qualys,Sentinel One,Sysmon,Trellix,Trend Micro,WatchGuard
0,Process Activity,Process Creation,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩
1,Process Activity,Process Termination,🟧,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟥,🟩,🟥,🎚️,🟥
2,Process Activity,Process Access,🟩,🟩,🟩,🟧,🟩,🟩,🟩,🟥,🟩,🟩,🟩,🟩,🟥
3,Process Activity,Image/Library Loaded,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩
4,Process Activity,Remote Thread Creation,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟥,🟩,🟩,🟩,🟩,🟩
5,Process Activity,Process Tampering Activity,🟧,🟩,❓,🟥,🟩,🟩,🟩,🟥,🟧,🟩,🟩,🟩,🟥
6,File Manipulation,File Creation,🟩,🟩,🟩,🟧,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟧
7,File Manipulation,File Opened,🟩,🟩,🟥,🟥,🟩,🟥,🟥,🟥,🟥,🟥,🟩,🟧,🟧
8,File Manipulation,File Deletion,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟩,🟥,🟥
9,File Manipulation,File Modification,🟩,🟩,🟥,🟩,🟩,🟩,🟩,🟥,🟩,🟥,🟩,🟩,🟥


list of sub-categories

In [58]:
columns_sub=new_df_x['Sub-Category'].unique()


In [59]:
columns=list(new_df_x.columns)
columns=columns[2:]

In [60]:
categories=new_df_x['Telemetry Feature Category'].unique()

In [61]:
dft=new_df_x.melt(id_vars=['Telemetry Feature Category','Sub-Category'], value_vars=columns)

In [62]:
dft.rename(columns={6:'product'}, inplace=True)

In [63]:
# single-value selection to create a dropdown
# selection1 = alt.selection_single(
#     name='Select',
#     fields=['Telemetry Feature Category'],
#     #init={'Telemetry Feature Category': 'Hash Algorithms'},
#     bind={'Telemetry Feature Category': alt.binding_select(options=categories)}
# )
  
#Binding selectiont to category legend
selec_leg = alt.selection_point(fields=['Telemetry Feature Category'], bind='legend')

#Selecting from key and values related to the status of the sub category
selec_keys=alt.selection_point(fields=['value'])

In [64]:
multi=[]
chart=alt.Chart(dft).mark_square(size=200,opacity=1).encode(
    alt.X('value:N', title=None),
    alt.Y('Sub-Category:N'),
    color='Telemetry Feature Category:N',
    tooltip='Telemetry Feature Category:N',
    opacity=alt.condition(selec_leg, alt.value(0.75), alt.value(0.05))).add_params(selec_leg,selec_keys).transform_filter(selec_leg&selec_keys)


keys=alt.Chart(new_df_legend).mark_text(
).encode(y=alt.Y('keys:N', title=None),text='value:N').add_params(selec_keys).transform_filter(selec_leg
)


result=(chart).facet(facet=alt.Facet('product:N', title=None, header=alt.Header(labelFontSize=15
        ))).properties(
    title={'text':['Compare products by sub-category'],
           'subtitle': ['Select in Legend by Status and Telemetry Feature Category', ' ']})
    # title='Comparing products by sub-category. Select in Legend to compare Telemetry Feature Categories'
    

In [65]:
inter_result=(result|keys).configure_axis(
    grid=False
).configure_view(
    stroke=None
).configure_title(fontSize=20)

In [66]:
inter_result

alt.HConcatChart(...)

In [67]:
chart.save('inter_result.html')